# Статистичні методи
Індивідуальне завдання 2

Токарєв Павло, МФ-51

## Підготовка даних
Текст узятий звідсіля: https://pamiec.pl/pa/biblioteka-cyfrowa/publikacje/53461,Polska-pod-okupacja-19391945-tom-1.html

Для початку ми видаляємо усі непотрібні знаки та переноси.

In [13]:
import math
import re
from typing import Dict, Iterable, List, Tuple
from collections import defaultdict
import unidecode as unidecode

bad_chars = "!(),-.0123456789:?[]\xad„”–"
with open("data/polish/1.txt") as file:
    text = file.read()
    text = "".join(c for c in text if c not in bad_chars).lower()
    text = re.sub(r"\s+", " ", text)
# sample
print(" ".join("\n"+word if i%20==19 else word for i,word in enumerate(text.split()) if i < 99))

ziemie polskie które w drugiej połowie września r dostały się pod okupację so wiecką nie były po prostu połową 
polski która tym tylko różniła się od tej zagarniętej przez niemców że leżała na wschodzie i że zajęli ją rosjanie 
był to poniekąd inny kraj już przed wojną tereny położone na wschód od wisły i bugu nazywano polską b a 
ziemie na zachód od linii obu tych rzek polską a była to zatem gorsza polska polska drugiej jakości skąd brała 
się ta pogarda dla kresów wschodnich ii rzeczypospolitej otóż ziemie te były gospodarczo i cywilizacyjnie zacofane z dominującym rolnictwem ponad


Усі літери тексту.

In [14]:
chars = sorted(set(text) - set(" "))
chars

['a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'y',
 'z',
 'ó',
 'ą',
 'ć',
 'ę',
 'ł',
 'ń',
 'ś',
 'ź',
 'ż']

Усі слова тексту.

In [15]:
words = [word for word in text.split() if len(word) > 1]
words[:20]

['ziemie',
 'polskie',
 'które',
 'drugiej',
 'połowie',
 'września',
 'dostały',
 'się',
 'pod',
 'okupację',
 'so',
 'wiecką',
 'nie',
 'były',
 'po',
 'prostu',
 'połową',
 'polski',
 'która',
 'tym']

Кількість слів.

In [16]:
len(words)

1491

Функція розбиття на n-грами.

In [17]:
def ngrams(words: Iterable[str], n: int) -> Dict[str, int]:
    d: Dict[str, int] = defaultdict(int)
    for word in words:
        if len(word) < n:
            continue
        for i in range(len(word)-(n-1)):
            d[word[i:i+n]] += 1
    return d

Отримані n-грами.

In [18]:
twograms = ngrams(words, 2)
list(twograms.items())[:20]

[('zi', 44),
 ('ie', 298),
 ('em', 66),
 ('mi', 106),
 ('po', 120),
 ('ol', 53),
 ('ls', 23),
 ('sk', 56),
 ('ki', 79),
 ('kt', 28),
 ('tó', 24),
 ('ór', 22),
 ('re', 45),
 ('dr', 6),
 ('ru', 19),
 ('ug', 6),
 ('gi', 13),
 ('ej', 74),
 ('oł', 9),
 ('ło', 47)]

In [19]:
trigrams = ngrams(words, 3)
list(trigrams.items())[:20]

[('zie', 31),
 ('iem', 26),
 ('emi', 7),
 ('mie', 30),
 ('pol', 34),
 ('ols', 23),
 ('lsk', 20),
 ('ski', 29),
 ('kie', 36),
 ('któ', 19),
 ('tór', 18),
 ('óre', 8),
 ('dru', 3),
 ('rug', 3),
 ('ugi', 5),
 ('gie', 6),
 ('iej', 27),
 ('poł', 6),
 ('oło', 4),
 ('łow', 9)]

Функції для обчислення матриці переходу та прогностичної здатності.

In [20]:
from collections import OrderedDict
from pandas import DataFrame

def prognozeability(matrix:List[Tuple[str, List[float]]]):
    pm = [
        (k, [1+sum(map(lambda p: p*math.log(p,2.0) if p > 0 else 0, l))/math.log(len(l), 2.0)])
        for k, l in matrix
    ]
    return DataFrame.from_dict(OrderedDict(pm), orient="index", columns=["prog. ability"])

def matrix(ngrams: Dict[str, int]):
    nmgram_amount = defaultdict(int)
    for k,v in ngrams.items():
        nmgram_amount[k[:-1]]+=v
    rev_ind = list(enumerate(sorted({key[:-1] for key in ngrams.keys()})))
    indexes = {key:i for i,key in rev_ind}
    m = [(rev_ind[i], [0 for _ in range(len(chars))]) for i in range(len(rev_ind))]
    for k,v in ngrams.items():
        c = k[-1]
        k = k[:-1]
        i = indexes[k]
        a = nmgram_amount[k]
        m[i][1][chars.index(c)] = v/a
    
    return DataFrame.from_dict(OrderedDict(m), orient="index", columns=chars), prognozeability(m)

## Результати
### 2-грами

In [21]:
two_matrix, twopa = matrix(twograms)
two_matrix

,a,b,c,d,e,f,g,h,i,j,...,z,ó,ą,ć,ę,ł,ń,ś,ź,ż
"(0, a)",0.001653,0.011570,0.120661,0.041322,0.000000,0.001653,0.023140,0.000000,0.009917,0.042975,...,0.019835,0.000000,0.000000,0.016529,0.000000,0.092562,0.021488,0.001653,0.000000,0.018182
"(1, b)",0.111111,0.000000,0.006944,0.000000,0.048611,0.000000,0.000000,0.000000,0.187500,0.000000,...,0.000000,0.000000,0.006944,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
"(2, c)",0.025157,0.000000,0.006289,0.000000,0.064990,0.000000,0.000000,0.253669,0.161426,0.062893,...,0.211740,0.008386,0.010482,0.000000,0.004193,0.000000,0.000000,0.000000,0.000000,0.000000
"(3, d)",0.052846,0.020325,0.008130,0.000000,0.056911,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.203252,0.020325,0.012195,0.000000,0.008130,0.012195,0.000000,0.000000,0.004065,0.000000
"(4, e)",0.000000,0.007042,0.103873,0.054577,0.000000,0.000000,0.091549,0.000000,0.003521,0.130282,...,0.031690,0.000000,0.000000,0.003521,0.000000,0.007042,0.005282,0.014085,0.001761,0.017606
"(5, f)",0.266667,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.266667,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
"(6, g)",0.163793,0.000000,0.000000,0.008621,0.000000,0.000000,0.000000,0.000000,0.112069,0.000000,...,0.008621,0.086207,0.000000,0.000000,0.000000,0.060345,0.000000,0.000000,0.000000,0.000000
"(7, h)",0.136364,0.000000,0.022727,0.000000,0.022727,0.000000,0.000000,0.000000,0.022727,0.000000,...,0.000000,0.068182,0.022727,0.000000,0.022727,0.000000,0.000000,0.000000,0.000000,0.000000
"(8, i)",0.122682,0.001427,0.084165,0.004280,0.425107,0.000000,0.001427,0.000000,0.011412,0.002853,...,0.011412,0.001427,0.019971,0.004280,0.055635,0.017118,0.007133,0.001427,0.000000,0.004280
"(9, j)",0.201220,0.012195,0.000000,0.006098,0.243902,0.000000,0.006098,0.000000,0.060976,0.000000,...,0.000000,0.000000,0.152439,0.000000,0.091463,0.000000,0.000000,0.006098,0.000000,0.000000


In [22]:
twopa

,prog. ability
"(0, a)",0.201573
"(1, b)",0.414531
"(2, c)",0.391072
"(3, d)",0.313432
"(4, e)",0.232360
"(5, f)",0.526227
"(6, g)",0.450148
"(7, h)",0.452535
"(8, i)",0.369145
"(9, j)",0.381566


### 3-грами

In [23]:
three_matrix, tripa = matrix(trigrams)
three_matrix

,a,b,c,d,e,f,g,h,i,j,...,z,ó,ą,ć,ę,ł,ń,ś,ź,ż
"(0, aa)",0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
"(1, ab)",0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.285714,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
"(2, ac)",0.000000,0.0,0.000000,0.000000,0.013889,0.0,0.0,0.222222,0.027778,0.250000,...,0.166667,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
"(3, ad)",0.176471,0.0,0.000000,0.000000,0.058824,0.0,0.0,0.000000,0.000000,0.000000,...,0.176471,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
"(4, af)",0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(445, żn)",0.181818,0.0,0.000000,0.000000,0.272727,0.0,0.0,0.000000,0.272727,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
"(446, żo)",0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
"(447, żs)",0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,...,1.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
"(448, ży)",0.000000,0.0,0.521739,0.130435,0.000000,0.0,0.0,0.000000,0.000000,0.086957,...,0.000000,0.0,0.0,0.043478,0.0,0.173913,0.0,0.0,0.0,0.0


In [24]:
tripa

,prog. ability
"(0, aa)",1.000000
"(1, ab)",0.556750
"(2, ac)",0.473167
"(3, ad)",0.440531
"(4, af)",1.000000
...,...
"(445, żn)",0.557673
"(446, żo)",1.000000
"(447, żs)",1.000000
"(448, ży)",0.601215
